The Following code is for scraping the Austrian online marketplace called Willhaben.at
The code scrapes through the website using selenium and soup.
For some reason Selenium is better at scraping the Price and Place info than Soup
The script can scrape listings as long as there is only one page for a keyword

The only missing feature is the ability to go through multiple pages

The code is written by matyasmarton

# Import the Libraries and define the base url and keywords

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [21]:
# The base_url contains the link, but the keywords are replaced with {0}
base_url = "https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz?keyword={0}&attribute_tree_level_0=&attribute_tree_level_1=&typedKeyword={0}"
# This is the list of keywords the script will go through
keywords = ['sony+fs','ursa+mini','red+camera','arri']

# Scraper

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Empty lists are created, these will be appended by the for loop

name_text = []
price_text = []
place_text = []
date_text = []
# Create the columns in the dataframe
df = pd.DataFrame( columns = ['Names','Prices','Dates','Places'])


try:
# First we format the url to contain the keywords    
    for keyword in keywords:
        url = base_url.format(keyword)
        driver = webdriver.Chrome(executable_path = r"C:\Users\Matyas\Documents\Untitled Folder\webdriver\chromedriver.exe")
# with requests, soup and selenium we open the browser and start scraping the site
        r = requests.get(url)
        driver.get(url)
        html = r.content
        soup = BeautifulSoup(html, "html5lib")
# Willhaben has a popup which needs to be accepted with Selenium
        button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, "didomi-notice-agree-button")))
        button.click()

# We start scraping the site first for name, then price, date and place.

        names = soup.find_all("span", {"itemprop":"name"})
# After the scraper obtained the data from the HTML markup we strip it from the tags and any whitespace 
        for name in names:
            name_stripped = name.text.strip()
# Then the script appends the scraped and clean data to an empty list
            name_text.append(name_stripped)
            

        prices = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "info-2-price")))
        for price in prices:
            price_stripped = price.text.strip()
            price_text.append(price_stripped)
        
       
            
        dates = soup.find_all("div",{"class":"bottom-content"})
        for date in dates:
            date_stripped = date.text.strip()
            date_text.append(date_stripped)

            
        places = WebDriverWait(driver, 15).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "address-lg.w-brk-ln-1 ")))
        for place in places:
            place_strip = place.text.strip()
            place_text.append(place_strip)
# After one website is done we print "visited {url of the website}"
        print(f"visited {url}")

        
finally:
# After the scraper got everything it pastes the data to the pandas dataframe
    df['Names'] = name_text
    df['Prices'] = price_text
    df['Dates'] = date_text
    df['Places'] = place_text
# Then selenium closes the webdriver
    driver.close()

visited https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz?keyword=sony+fs&attribute_tree_level_0=&attribute_tree_level_1=&typedKeyword=sony+fs
visited https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz?keyword=ursa+mini&attribute_tree_level_0=&attribute_tree_level_1=&typedKeyword=ursa+mini
visited https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz?keyword=red+camera&attribute_tree_level_0=&attribute_tree_level_1=&typedKeyword=red+camera
visited https://www.willhaben.at/iad/kaufen-und-verkaufen/marktplatz?keyword=arri&attribute_tree_level_0=&attribute_tree_level_1=&typedKeyword=arri
